In [94]:
%pip install scipy
%pip install yfinance 
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize # Import the minimize function



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [95]:
mmm = yf.download(tickers='MMM', period='6mo')
wbd = yf.download(tickers='WBD', period='6mo')
jnj = yf.download(tickers='JNJ', period='6mo')
aapl =yf.download(tickers='AAPL',period='6mo')
hon = yf.download(tickers='HON', period='6mo')
nvda = yf.download(tickers='NVDA', period='6mo')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [96]:
mmm['Return'] = mmm['Adj Close'].pct_change()
wbd['Return'] = wbd['Adj Close'].pct_change()
jnj['Return'] = jnj['Adj Close'].pct_change()
aapl['Return'] = aapl['Adj Close'].pct_change()
hon['Return'] = hon['Adj Close'].pct_change()
nvda['Return'] = nvda['Adj Close'].pct_change()
returns = pd.DataFrame({
    '3M': mmm['Return'],
    'WBD': wbd['Return'],
    'JNJ': jnj['Return'],
    'AAPL': aapl['Return'],
    'HONEY': hon['Return'],
    'Nvidia': nvda['Return']
})

# Drop any rows with NaN values
returns.dropna(inplace=True)
print(returns)
# Calculate the variance-covariance matrix
C = returns.cov()

#print(C)


                  3M       WBD       JNJ      AAPL     HONEY    Nvidia
Date                                                                  
2024-01-09  0.002205 -0.032456  0.000619 -0.002263 -0.005082  0.016975
2024-01-10 -0.007333 -0.005440  0.001485  0.005671  0.003819  0.022770
2024-01-11  0.005725 -0.039198 -0.004448 -0.003222 -0.003607  0.008684
2024-01-12 -0.007345  0.010436  0.007695  0.001778 -0.002826 -0.002043
2024-01-16 -0.004439 -0.028169 -0.011516 -0.012317 -0.016559  0.030561
...              ...       ...       ...       ...       ...       ...
2024-06-28 -0.009499  0.012245  0.002469 -0.016254 -0.003918 -0.003629
2024-07-01 -0.015461 -0.045699  0.001916  0.029105 -0.012129  0.006152
2024-07-02  0.010039  0.014084 -0.002800  0.016240  0.010808 -0.013113
2024-07-03  0.000000  0.004167 -0.002328  0.005811 -0.003611  0.045732
2024-07-05 -0.002952 -0.002766  0.005422  0.021620 -0.001035 -0.019099

[123 rows x 6 columns]


In [97]:
#METHOD- 1 IMPLEMTATION

w = np.ones(6)/6   #initial guess for weight matrix
C = np.array(C)
def obj_function(w, C):
    P = C @ w
    MCR = P/np.sqrt(w.T @ C @ w)
    f = 0.0
    for i in range(len(w)):
        for j in range(len(w)):
            f += (w[i] * MCR[i] - w[j] * MCR[j]) ** 2
    return f

def objective(w):
    return obj_function(w, C)

# Constraints
constraints = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Equality constraint: sum(w) = 1
    {'type': 'ineq', 'fun': lambda w: w}             # Inequality constraint: w >= 0
)

# Bounds
bounds = [(0, 1) for _ in range(C.shape[1])]



# Optimize
result = minimize(objective, w, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': True})

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)
print("Objective function value at optimal weights:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.150908471199883e-05
            Iterations: 1
            Function evaluations: 7
            Gradient evaluations: 1
Optimal weights: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
Objective function value at optimal weights: 5.150908471199883e-05
Success: True
Message: Optimization terminated successfully


In [98]:
#METHOD-2 IMPLEMNTATION
w = np.ones(6)/6   #initial guess for weight matrix
C = np.array(C)
def obj_function(w, C):
    return np.sqrt(w.T @ C @ w)


def objective(w):
    return obj_function(w, C)

# Constraints
constraints = (
 {'type': 'ineq', 'fun': lambda w: np.sum(np.log(w)) + 2 },  #I have taken c = -2, you can take other arbitrary constant which gives a feasible soln
    {'type': 'ineq', 'fun': lambda w: w}             # Inequality constraint: w >= 0
)

# Bounds
bounds = [(0, 1) for _ in range(C.shape[1])]



# Optimize
result = minimize(objective, w, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': True})

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)
print("Objective function value at optimal weights:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

y = np.array(optimal_weights)
#Normalise
weights = y/np.sum(y)
print(weights)   #Final weight matrix

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.034746261806249464
            Iterations: 19
            Function evaluations: 134
            Gradient evaluations: 19
Optimal weights: [0.70019818 0.445167   1.         0.79774612 1.         0.54425485]
Objective function value at optimal weights: 0.034746261806249464
Success: True
Message: Optimization terminated successfully
[0.15603767 0.09920452 0.22284787 0.17777602 0.22284787 0.12128604]


In [99]:
returns

,3M,WBD,JNJ,AAPL,HONEY,Nvidia
Date,,,,,,
2024-01-09,0.002205,-0.032456,0.000619,-0.002263,-0.005082,0.016975
2024-01-10,-0.007333,-0.005440,0.001485,0.005671,0.003819,0.022770
2024-01-11,0.005725,-0.039198,-0.004448,-0.003222,-0.003607,0.008684
2024-01-12,-0.007345,0.010436,0.007695,0.001778,-0.002826,-0.002043
2024-01-16,-0.004439,-0.028169,-0.011516,-0.012317,-0.016559,0.030561
...,...,...,...,...,...,...
2024-06-28,-0.009499,0.012245,0.002469,-0.016254,-0.003918,-0.003629
2024-07-01,-0.015461,-0.045699,0.001916,0.029105,-0.012129,0.006152
2024-07-02,0.010039,0.014084,-0.002800,0.016240,0.010808,-0.013113


In [100]:
returns.shape[0]

123

In [101]:
def annualize_rets(r,periods_per_year):
    compounded_growth=(1+r).prod()
    n_periods= r.shape[0]
    return compounded_growth**(periods_per_year/n_periods)-1

def portfolio_return(weights,returns):
    ''' Weights -> Returns'''
    return weights.T @ returns

def portfolio_var(weights, covmat):
    ''' Weightd->Vol'''
    return (weights.T @ covmat @ weights)**0.5


In [102]:
#calculating expected returns
er=annualize_rets(returns,12)
er

3M        0.012823
WBD      -0.043713
JNJ      -0.007986
AAPL      0.019832
HONEY     0.005572
Nvidia    0.089537
dtype: float64

In [103]:
#Calculating Portfolio Returns and Voltility

r_final= portfolio_return(weights, er)
variance= portfolio_var(weights,C)

print("The returns and variance for the Risk parity weights",r_final,variance)

0.011511483805127894 0.0077431305243032215


#Assigning market cap weights


In [117]:
mmm_c=yf.Ticker("MMM").info['marketCap']
wbd_c=yf.Ticker("WBD").info['marketCap']
jnj_c=yf.Ticker("JNJ").info['marketCap']
aapl_c=yf.Ticker("AAPL").info['marketCap']
hon_c=yf.Ticker("HON").info['marketCap']
nvda_c=yf.Ticker("NVDA").info['marketCap']
l=np.array([mmm_c,wbd_c,jnj_c,aapl_c,hon_c,nvda_c])
print(l)
s=l.sum()
print(s)
mc_weights=l/s 
print(mc_weights)
mc_weights.sum()

[  56066539520   17666734080  352530497536 3470720237568  138207723520
 3095204200448]
7130395932672
[0.00786303 0.00247767 0.04944052 0.48675    0.0193829  0.43408588]


1.0

In [123]:
er_mc=annualize_rets(returns,12)
mc_rets=portfolio_return(mc_weights,er_mc)
mc_var=portfolio_var(mc_weights,C)
print("Returns and Variance on Market Cap Weights =", mc_rets,mc_var)
print("The returns and variance for the Risk parity weights",r_final,variance)

Returns and Variance on Market Cap Weights = 0.04822561220518394 0.016606843164719273
The returns and variance for the Risk parity weights 0.011511483805127894 0.0077431305243032215
